In [1]:
!git clone https://github.com/ChitranjanUpadhayay/ML_Projects.git/

Cloning into 'ML_Projects'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 13 (delta 0), reused 3 (delta 0), pack-reused 10
Receiving objects: 100% (13/13), 41.86 MiB | 13.70 MiB/s, done.
Updating files: 100% (6/6), done.


In [2]:
import pandas as pd
import numpy as np

In [3]:
fake = pd.read_csv('/content/ML_Projects/Datasets/Fake News/Fake.csv')
true = pd.read_csv('/content/ML_Projects/Datasets/Fake News/True.csv')

In [4]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [6]:
true['label'] = 1
fake['label'] = 0

In [7]:
true.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [8]:
fake.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [9]:
news = pd.concat([true, fake], axis=0)

In [10]:
news.tail()

,title,text,subject,date,label
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",0


In [11]:
news.isnull().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [12]:
news = news.drop(['title', 'subject', 'date'], axis=1)

In [13]:
news.head()

,text,label
0,WASHINGTON (Reuters) - The head of a conservat...,1
1,WASHINGTON (Reuters) - Transgender people will...,1
2,WASHINGTON (Reuters) - The special counsel inv...,1
3,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,1


In [14]:
# shuffling the data
news = news.sample(frac=1)
news.head()

,text,label
13926,MOSCOW (Reuters) - Russia s Defence Ministry s...,1
19071,"BRIGHTON, England (Reuters) - Britain s opposi...",1
19433,Hey Michelle Guess what? For the first time in...,0
9354,As more and more women line up to tell their s...,0
4683,When a Donald Trump minion whined about Hillar...,0


In [15]:
# Arranging the index problem
news.reset_index(inplace=True)
news.head()

,index,text,label
0,13926,MOSCOW (Reuters) - Russia s Defence Ministry s...,1
1,19071,"BRIGHTON, England (Reuters) - Britain s opposi...",1
2,19433,Hey Michelle Guess what? For the first time in...,0
3,9354,As more and more women line up to tell their s...,0
4,4683,When a Donald Trump minion whined about Hillar...,0


In [16]:
news.drop(['index'], axis=1, inplace=True)

In [17]:
news.head()

,text,label
0,MOSCOW (Reuters) - Russia s Defence Ministry s...,1
1,"BRIGHTON, England (Reuters) - Britain s opposi...",1
2,Hey Michelle Guess what? For the first time in...,0
3,As more and more women line up to tell their s...,0
4,When a Donald Trump minion whined about Hillar...,0


In [18]:
import re

**Punctuation**: a set of symbols used to separate and clarify the meaning of sentences and written elements



In [19]:
def wordopt(text):
  # into lowercase
  text = text.lower()

  # remove urls
  text = re.sub(r'https?://\S+|www\.\S+', '', text)

  # remove html tags
  text = re.sub(r'<.*?>', '', text)

  # remove punctuations
  text = re.sub(r'[^\w\s]', '', text)

  # remove digits
  text = re.sub(r'\d', '', text)

  # remove newline characters
  text = re.sub(r'\n', '', text)

  return text

In [20]:
news['text'] = news['text'].apply(wordopt)

In [21]:
news['text']

0        moscow reuters  russia s defence ministry said...
1        brighton england reuters  britain s opposition...
2        hey michelle guess what for the first time in ...
3        as more and more women line up to tell their s...
4        when a donald trump minion whined about hillar...
                               ...                        
44893    a snowflake came undone when she had to obey p...
44894    london reuters  british prime minister theresa...
44895    manila reuters  stepping up a campaign against...
44896    moscow reuters  the kremlin said on wednesday ...
44897    what can we expect in  inflated markets inside...
Name: text, Length: 44898, dtype: object

In [22]:
x = news['text']
y = news['label']

In [23]:
x.shape

(44898,)

In [24]:
y.shape

(44898,)

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization = TfidfVectorizer()
vectorization.fit(x)
x = vectorization.transform(x)

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
xv_train, xv_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [28]:
xv_train.shape

(31428, 208652)

In [29]:
xv_test.shape

(13470, 208652)

In [30]:
y_train.shape

(31428,)

In [31]:
y_test.shape

(13470,)

Tfid vectorizer in used to interpret textual data into numerical data.

In [32]:
xv_train

<31428x208652 sparse matrix of type '<class 'numpy.float64'>'
	with 6458370 stored elements in Compressed Sparse Row format>

In [33]:
xv_test

<13470x208652 sparse matrix of type '<class 'numpy.float64'>'
	with 2760785 stored elements in Compressed Sparse Row format>

In [34]:
from sklearn.linear_model import LogisticRegression

In [35]:
lr = LogisticRegression()

In [36]:
lr.fit(xv_train, y_train)

LogisticRegression()

In [37]:
pred_lr = lr.predict(xv_test)

In [38]:
lr.score(xv_test, y_test)

0.9888641425389755

In [39]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      7087
           1       0.99      0.99      0.99      6383

    accuracy                           0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470



In [40]:
from sklearn.tree import DecisionTreeClassifier

In [41]:
dtc = DecisionTreeClassifier()

In [42]:
dtc.fit(xv_train, y_train)

DecisionTreeClassifier()

In [43]:
pred_dtc = dtc.predict(xv_test)

In [44]:
dtc.score(xv_test, pred_dtc)

1.0

In [45]:
print(classification_report(y_test, pred_dtc))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7087
           1       1.00      1.00      1.00      6383

    accuracy                           1.00     13470
   macro avg       1.00      1.00      1.00     13470
weighted avg       1.00      1.00      1.00     13470



In [46]:
from sklearn.ensemble import RandomForestClassifier

In [47]:
rfc = RandomForestClassifier()

In [48]:
rfc.fit(xv_train, y_train)

RandomForestClassifier()

In [49]:
pred_rfc = rfc.predict(xv_test)

In [50]:
rfc.score(xv_test, pred_rfc)

1.0

In [51]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      7087
           1       0.99      0.99      0.99      6383

    accuracy                           0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470



In [52]:
from sklearn.ensemble import GradientBoostingClassifier

In [53]:
gbc = GradientBoostingClassifier()

In [54]:
gbc.fit(xv_train, y_train)

KeyboardInterrupt: 

In [ ]:
pred_gbc = gbc.predict(xv_test)

In [ ]:
gbc.score(xv_test, pred_gbc)

In [ ]:
print(classification_report(y_test, pred_gbc))

In [55]:
def output_label(n):
  if n==0:
    return "This is fake news"
  elif n==1:
    return "This is true news"

In [56]:
def manual_testing(news):
  testing_news = {'text': [news]}
  new_def_test = pd.DataFrame(testing_news)
  new_def_test['text'] = new_def_test['text'].apply(wordopt)
  new_x_test = new_def_test['text']
  new_xv_test = vectorization.transform(new_x_test)
  pred_lr = lr.predict(new_xv_test)
  pred_rfc = rfc.predict(new_xv_test)
  pred_dtc = dtc.predict(new_xv_test)
  return "\n\n LR Prediction: {} \nDTC Prediction: {} \nRFC Prediction: {}".format(output_label(pred_lr[0]),
                                                                                   output_label(pred_dtc[0]),
                                                                                   output_label(pred_rfc[0])
                                                                                   )


In [57]:
news_article = str(input())

"Scientists Discover New Species of Deep-Sea Octopus with Bioluminescent Tentacles"


In [58]:
manual_testing(news_article)

'\n\n LR Prediction: This is fake news \nDTC Prediction: This is fake news \nRFC Prediction: This is fake news'

In [59]:
news_article1 = str(input())

"World Health Organization Declares Ebola Outbreak in West Africa a Public Health Emergency"


In [60]:
manual_testing(news_article1)

'\n\n LR Prediction: This is fake news \nDTC Prediction: This is fake news \nRFC Prediction: This is fake news'

In [61]:
news_article2 = str(input())

"NASA's Perseverance Rover Successfully Collects First Mars Rock Samples for Return to Earth"


In [62]:
manual_testing(news_article2)

'\n\n LR Prediction: This is fake news \nDTC Prediction: This is fake news \nRFC Prediction: This is fake news'